In [1]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, Dense, Flatten

print(f'Tensorflow version: {tf.__version__}')

Tensorflow version: 2.9.1


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype('float32')
x_test = x_test[..., tf.newaxis].astype('float32')

In [3]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [4]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel()

In [5]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [6]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [7]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different behavior during training versus inference (e.g. Dropout)
        predictions = model(images, training=True)
        loss = loss_fn(labels, predictions)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [8]:
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different behavior during training versus inference (e.g. Dropout)
    predictions = model(images, training=False)
    t_loss = loss_fn(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [9]:
EPOCHS = 5

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
        f'Epoch: {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test loss: {test_loss.result()}, '
        f'Test accuracy: {test_accuracy.result() * 100}'
    )

Epoch: 1, Loss: 0.1330355405807495, Accuracy: 96.0816650390625, Test loss: 0.05471613630652428, Test accuracy: 98.18000030517578
Epoch: 2, Loss: 0.04106901213526726, Accuracy: 98.67666625976562, Test loss: 0.046377893537282944, Test accuracy: 98.44999694824219
Epoch: 3, Loss: 0.022117894142866135, Accuracy: 99.31666564941406, Test loss: 0.05340142920613289, Test accuracy: 98.29000091552734
Epoch: 4, Loss: 0.013343053869903088, Accuracy: 99.56832885742188, Test loss: 0.05849776044487953, Test accuracy: 98.33999633789062
Epoch: 5, Loss: 0.008514135144650936, Accuracy: 99.73666381835938, Test loss: 0.061201054602861404, Test accuracy: 98.30999755859375
